<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/%E8%BD%B4%E6%89%BF%E6%A3%80%E6%B5%8BCNN%E4%BB%A3%E7%A0%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.io import loadmat
import numpy as np
import os
import pandas as pd
from sklearn.decomposition import PCA 
from pandas.core.frame import DataFrame
import torch
import torch.utils.data.dataset as Dataset
import torch.utils.data.dataloader as DataLoader

import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

import torch.optim as optim


path = r'data'

In [ ]:
#读数据名
filenames = os.listdir(r'data')
files = {}
#读取每个数据
for i in filenames:
  #路径
  file_path = os.path.join(r'data', i)
  #加载
  file = loadmat(file_path)
  #一个文件中不同类型数据
  file_keys = file.keys()
  for key in file_keys:
    #选择；有DE的数据
    if 'DE' in key:
      #整理成一行一维数据
      files[i] = file[key].ravel()

In [ ]:
#一个数据1.400
length=400
#数据取值步长400，就是第一个1-400，第二个400-800
step=400
#标签从0开始
label=0
#建立数据集空白的
#训练集
df_train = pd.DataFrame(columns=('data', 'label','name'))
#验证
df_val = pd.DataFrame(columns=('data', 'label','name'))
#测试
df_test = pd.DataFrame(columns=('data', 'label','name'))
keys = files.keys()
#keys是不同标签的分类
for i in keys:
  #全部
  slice_data = files[i]
  #长度
  all_lenght = len(slice_data)
  #训练结束节点
  train_end_index = int(all_lenght*0.6)
  #验证结束节点
  val_end_index = int(all_lenght*0.8)
  start_index=0
  #训练结束节点之前切数据成400
  while start_index+length <= train_end_index:
      #切
      sample = slice_data[start_index:start_index + length]
      #加400
      start_index+=step
      #加到数据集上
      df_train=df_train.append([{'data':sample,'label':label,'name':i}], ignore_index=True)
  
  #注释同上
  start_index=train_end_index    
  while start_index+length <= val_end_index:
      sample = slice_data[start_index:start_index + length]
      start_index+=step
      
      df_val=df_val.append([{'data':sample,'label':label,'name':i}], ignore_index=True)
      
  start_index=val_end_index    
  while start_index+length <= all_lenght:
      sample = slice_data[start_index:start_index + length]
      start_index+=step      
      df_test=df_test.append([{'data':sample,'label':label,'name':i}], ignore_index=True)
  
  label+=1

打乱数据集

In [ ]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)
df_test = shuffle(df_test)
df_val = shuffle(df_val)
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)

数据可视化（你不需要注释吧）

In [ ]:
noramlarry=np.expand_dims(df_train['data'][0],axis=0)
train_nomal=df_train[df_train['label']==1]
for arry in train_nomal['data']:
  arry=np.expand_dims(arry,axis=0)
  noramlarry=np.append(noramlarry, arry, axis=0)

gundongarry=np.expand_dims(df_train['data'][0],axis=0)
train_gundong=df_train[df_train['label']==2]
for arry in train_gundong['data']:
  arry=np.expand_dims(arry,axis=0)
  gundongarry=np.append(gundongarry, arry, axis=0)

outarry=np.expand_dims(df_train['data'][0],axis=0)
train_out=df_train[df_train['label']==3]
for arry in train_out['data']:
  arry=np.expand_dims(arry,axis=0)
  outarry=np.append(outarry, arry, axis=0)

inarry=np.expand_dims(df_train['data'][0],axis=0)
train_in=df_train[df_train['label']==0]
for arry in train_in['data']:
  arry=np.expand_dims(arry,axis=0)
  inarry=np.append(inarry, arry, axis=0)  

In [ ]:
allarry=np.expand_dims(df_train['data'][0],axis=0)
for arry in df_train['data']:
  arry=np.expand_dims(arry,axis=0)
  allarry=np.append(allarry, arry, axis=0)

In [ ]:
sklearn.decomposition.PCA
pca = PCA(n_components=3)
pca=pca.fit(allarry)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(1, figsize=(8, 8))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
plt.cla()

pca1=pca.fit(noramlarry)
X_1 = pca1.transform(noramlarry)
pca2=pca.fit(gundongarry)
X_2 = pca2.transform(gundongarry)

pca3=pca.fit(inarry)
X_3 = pca3.transform(inarry)
pca4=pca.fit(outarry)
X_4 = pca4.transform(outarry)
x1=ax.scatter(X_1[:, 0], X_1[:, 1], X_1[:, 2],c='#1f77b4',s=10, marker="o", edgecolor='face')
x2=ax.scatter(X_2[:, 0], X_2[:, 1], X_2[:, 2],c='#2ca02c',s=10, marker="x", edgecolor='face')
x3=ax.scatter(X_3[:, 0], X_3[:, 1], X_3[:, 2],c='#ff7f0e',s=10, marker="s", edgecolor='face')
x4=ax.scatter(X_4[:, 0], X_4[:, 1], X_4[:, 2],c='#8c564b',s=10, marker="v", edgecolor='face')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])
ax.legend([x1, x2, x3, x4], ['Normal','Ball','Outer Race','InnerRace'])
plt.show()

建立Dataset

用于原始数据的分类数据集

In [ ]:
#用于原始数据的分类数据集
class subDataset(Dataset.Dataset):
    #初始化，定义数据内容和标签
    def __init__(self, Datadf):
        self.Data = Datadf['data']
        self.Label = Datadf['label']
    #返回数据集大小
    def __len__(self):
        return len(self.Data)
    #得到数据内容和标签
    def __getitem__(self, index):
        #datasize=(1,400)
        data = torch.Tensor(self.Data[index]).unsqueeze(0)
        label=self.Label[index]
        return data, label

用于计算FID的数据集存储为20*20的图像

In [ ]:

class imgDataset(Dataset.Dataset):
    #初始化，定义数据内容和标签
    def __init__(self, Datadf):
        #内容
        self.Data = Datadf['data']
        #标签
        self.Label = Datadf['label']

    #返回数据集大小
    def __len__(self):
        return len(self.Data)

    #得到数据内容和标签
    def __getitem__(self, index):
        #datasize=(1,20,20)
        data = torch.Tensor(self.Data[index]).reshape(20,20)
        label=self.Label[index]
        return data, label

In [ ]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #输入为1，400
        #进行BatchNorm
        self.batchnorm1= nn.BatchNorm1d(1)
        #输入为1，400，一维卷积卷积核大小3，padding 1 步长2 输出为8，200
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=8, kernel_size=3,padding=1,stride=2)
        
        
        #进行BatchNorm
        self.batchnorm3= nn.BatchNorm1d(8)
        #输入为8，200，一维卷积卷积核大小3，padding 1 步长2 输出为1，100
        self.conv3 = nn.Conv1d(in_channels=8, out_channels=1, kernel_size=3,padding=1,stride=2)


        #输入为1，100 输出为1，10
        self.fc1 = nn.Linear(in_features=100,out_features=10)
        #输入为1，100 输出为1，4
        self.fc2 = nn.Linear(in_features=10,out_features=4)

    def forward(self, x):
        x= self.batchnorm1(x)
        #采用relu作为激活函数
        x = F.relu(self.conv2(x))
        x= self.batchnorm3(x)
        #采用relu作为激活函数
        x = F.relu(self.conv3(x))
        #改变形状为1，100
        x = x.view(-1, 100)             
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

训练过程

In [1]:
def train(epoch):
    #首先让网络能训练
    net.train()
    #从dataloader里面读数据
    for data in tqdm(dataloader, leave=False, total=len(dataloader)):
        #输入为1，400
        inputs=data[0]
        #目标标签是0-3之间的数字
        targets=data[1]

        #这些放在GPU上
        inputs=inputs.to(device)
        targets=targets.to(device)

        #每次喂入数据前，都需要将梯度清零        
        optimizer.zero_grad()

        #得到输出结果
        outputs = net(inputs)

        #计算loss
        loss = criterion(outputs, targets)

        #传回反向梯度
        loss.backward()
        #梯度传回，利用优化器将参数更新
        optimizer.step()
    
    #打印loss 
    print(loss)
    
    #为了可视化返回loss的值
    return(loss.item())

In [ ]:
test_loss={}
def test(epoch):

    #最优的准确度
    global best_acc
    #让神经网路进行验证
    net.eval()
    
    #初始化
    test_loss = 0
    correct = 0
    total = 0
    best_acc=0


    #因为是测试，因此禁止梯度
    with torch.no_grad():

        #从dataloader里面读数据
        for batch_idx, (inputs, targets) in enumerate(testloader):

            #这些放在GPU上
            inputs, targets = inputs.to(device), targets.to(device)

            #得到输出结果
            outputs = net(inputs)
            
            #计算损失
            loss = criterion(outputs, targets)            
            test_loss += loss.item()

            #用于得到准确度
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()


    #计算准确度
    acc = 100. * correct / total
    print("Now acc is {}".format(acc))
    #保存准确度最好的值
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
          os.mkdir('checkpoint')
          torch.save(state, './checkpoint/ckpt.t7')
        best_acc = acc
    return acc 


In [ ]:
#使用GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#神经网络放在GPU上
net = net.to(device)

#采用CrossEntropyLoss作为损失函数
criterion = nn.CrossEntropyLoss()

#采用Adam作为优化器，学希率为0.001
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
train_losses = []
accurate = []

#迭代100次
for epoch in range(0,100):

        #保存loss
        train_losses.append(train(epoch))
        accurate.append(test(epoch))

In [ ]:
#画图可视化
plt.figure(figsize=(10,5))
plt.title("Train Loss During Training")
plt.plot(train_losses,label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")

plt.legend()
plt.savefig('./CNNloss.jpg')
plt.show()

plt.figure(figsize=(10,5))
plt.title("Train Loss During Training")
plt.plot(train_losses,label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")

plt.legend()
plt.savefig('./GANCNNloss.jpg')
plt.show()

In [ ]:
#注释参考test()
def test_data():
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    best_acc=0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            print(predicted)
            print(targets)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = 100. * correct / total
    print("Now acc is {}".format(acc))